The current notebook requires TF2 to run (see [databricks 6.1ML Runtime for how to install TF2](https://docs.databricks.com/applications/deep-learning/single-node-training/tensorflow.html#install-tensorflow-20-on-dbr-61-ml))

TODO
- [] Add tf.summary to monitor all statistics in tensorboard (see [tutorial](https://www.tensorflow.org/tensorboard/migrate))
- [x] Change how inference is done: currently no mask is used when validation accuracy is measured
- [x] Add an option to use MSE LOSS
- [x] Add a function to load initializations from the teacher model to the student model

In [3]:
import tensorflow as tf
import tensorflow_datasets
from transformers import *
import tensorflow_hub as hub
import sys

/databricks/python/lib/python3.7/site-packages/botocore/vendored/requests/packages/urllib3/_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 from collections import Mapping, MutableMapping

In [4]:
print(tf.__version__)
print(hub.__version__)

2.0.0
0.7.0

Define DistillBertConfig as a subclass of BertConfig and DistilConfig to incorporate the following parameters

In [6]:
class DistilMyBertConfig(PretrainedConfig):
    def __init__(self,
                 vocab_size_or_config_json_file=33333,
                 num_classes=2,
                 distill_temperature=2.0,
                 task_balance=0.5,
                 max_seq_len=128,
                 epoch = 5,
                 learning_rate=5e-4,
                 adam_epsilon=1e-6,
                 max_grad_norm=5.0,
                 **kwargs):
        super(PretrainedConfig, self).__init__(*kwargs)
        
        if isinstance(vocab_size_or_config_json_file, str) or (sys.version_info[0] == 2
                        and isinstance(vocab_size_or_config_json_file, unicode)):
            with open(vocab_size_or_config_json_file, "r", encoding='utf-8') as reader:
                json_config = json.loads(reader.read())
            for key, value in json_config.items():
                self.__dict__[key] = value
        elif isinstance(vocab_size_or_config_json_file, int):
            self.num_classes=num_classes
            self.distill_temperature = distill_temperature
            self.task_balance = task_balance
            self.max_seq_len = max_seq_len
            self.epoch = epoch
            self.learning_rate = learning_rate
            self.adam_epsilon = adam_epsilon
            self.max_grad_norm = max_grad_norm

In [7]:
# Prepare Config
config = DistilMyBertConfig()

In [8]:
# Get stuent and teacher nn architecture

student_config_class, student_model_class, student_tokenizer_class = DistilBertConfig, TFDistilBertForSequenceClassification, DistilBertTokenizer
teacher_config_class, teacher_model_class, teacher_tokenizer_class = BertConfig, TFBertForSequenceClassification, BertTokenizer

# Load teacher model from checkpoints, but freeze the teacher layers
teacher = teacher_model_class.from_pretrained("/dbfs/ml/judith/transformers/mrpc/1")#, output_hidden_states=True)
teacher_tokenizer = teacher_tokenizer_class.from_pretrained('/dbfs/ml/judith/transformers/mrpc/1') 

# # Load student model from config
# # Student model no longer has token type embedding and the position embedding
# student_config_path = "/dbfs/ml/judith/transformers/distilbert-base-uncased.json"
# stu_architecture_config = student_config_class.from_pretrained(student_config_path)
# stu_architecture_config.sinusoidal_pos_embds = False
# stu_architecture_config.n_layers = 1
# student = student_model_class(stu_architecture_config)
# print("Is the student weights are empty before training? ", student.weights==[])

Is the student weights are empty before training? True

In [9]:
teacher_config = teacher_config_class.from_pretrained('bert-base-uncased')
teacher_config.num_hidden_layers = 1
student = teacher_model_class(teacher_config)
print("Is the teacher model weights are empty before training? ", student.weights==[])
student(student.dummy_inputs, training=False)

Is the teacher model weights are empty before training? True
Out[113]: (<tf.Tensor: id=606639, shape=(3, 2), dtype=float32, numpy=
 array([[-0.17411079, -0.03499376],
 [-0.28216106, -0.16699678],
 [ 0.02154749, -0.17930067]], dtype=float32)>,)

In [10]:
student.load_weights('/tmp/experiment3/teacher/tf_model.h5', by_name=True)
student(student.dummy_inputs, training=False)

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-145929> in <module> 
 ----> 1 student . load_weights ( '/tmp/experiment3/teacher/tf_model.h5' , by_name = True ) 
 2 student ( student . dummy_inputs , training = False ) 

 /databricks/python/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training.py in load_weights (self, filepath, by_name) 
 179 raise ValueError('Load weights is not yet supported with TPUStrategy '
 180 'with steps_per_run greater than 1.')
 --> 181 return super ( Model , self ) . load_weights ( filepath , by_name ) 
 182 
 183 @ trackable . no_automatic_dependency_tracking

 /databricks/python/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/network.py in load_weights (self, filepath, by_name) 
 1173 f = f [ 'model_weights' ] 
 1174 if by_name : 
 -> 1175 saving . load_weights_from_hdf5_group_by_name ( f , self . layers ) 
 1176 else : 
 1177 saving . load_weights_from_hdf5_group ( f , self . layers ) 

 /databricks/python/lib/python3.7/site-packages/tensorflow_core/python/keras/saving/hdf5_format.py in load_weights_from_hdf5_group_by_name (f, layers) 
 749 '") expects ' + str ( len ( symbolic_weights ) ) + 
 750 ' weight(s), but the saved weights' + ' have ' + 
 --> 751 str(len(weight_values)) + ' element(s).')
 752 # Set values. 
 753 for i in range ( len ( weight_values ) ) : 

 ValueError : Layer #0 (named "bert") expects 23 weight(s), but the saved weights have 199 element(s).

In [11]:
print(student(student.dummy_inputs, training=False))
layer_mapping = {0:0}
extract_weights_from_teacher(teacher, student, layer_mapping)
# student(student.dummy_inputs, training=False)

(<tf.Tensor: id=606877, shape=(3, 2), dtype=float32, numpy=
array([[-0.17411079, -0.03499376],
 [-0.28216106, -0.16699678],
 [ 0.02154749, -0.17930067]], dtype=float32)>,)

In [12]:
student(student.dummy_inputs, training=False)

Out[115]: (<tf.Tensor: id=607115, shape=(3, 2), dtype=float32, numpy=
 array([[-0.17411079, -0.03499376],
 [-0.28216106, -0.16699678],
 [ 0.02154749, -0.17930067]], dtype=float32)>,)

In [13]:
# save pre-trained weights
teacher = teacher_model_class.from_pretrained('bert-base-uncased')
teacher.save_pretrained('/tmp/experiment3/teacher')

# build teacher architecture
teacher_config = teacher_config_class.from_pretrained('bert-base-uncased')
teacher = teacher_model_class(teacher_config)
teacher(teacher.dummy_inputs, training=False)

# initialize teacher from pre-trained weights
teacher.load_weights('/tmp/experiment3/teacher/tf_model.h5')
teacher(teacher.dummy_inputs, training=False)

Out[80]: (<tf.Tensor: id=364163, shape=(3, 2), dtype=float32, numpy=
 array([[-0.25553346, 0.01401245],
 [-0.18620504, 0.12456853],
 [-0.05898177, 0.18377906]], dtype=float32)>,)

In [14]:
# num_hidden_layers = 1
# training=True

# teacher = teacher_model_class.from_pretrained('bert-base-uncased')

# input_ids = tf.keras.layers.Input(shape=(config.max_seq_len,), dtype=tf.int32,
#                                        name="input_word_ids")
# attention_mask = tf.keras.layers.Input(shape=(config.max_seq_len,), dtype=tf.int32,
#                                    name="attention_mask")
# token_type_ids = tf.keras.layers.Input(shape=(config.max_seq_len,), dtype=tf.int32,
#                                     name="token_type_ids")

# extended_attention_mask = attention_mask[:, tf.newaxis, tf.newaxis, :]
# extended_attention_mask = tf.cast(extended_attention_mask, tf.float32)
# extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0
# head_mask = [None] * num_hidden_layers
# position_ids = None

# embedding_output = teacher.bert.embeddings([input_ids, position_ids, token_type_ids], training=training)
# encoder_outputs = teacher.bert.encoder.layer[0]([embedding_output, extended_attention_mask, head_mask], training=training)

# sequence_output = encoder_outputs[0]
# pooled_output = teacher.bert.pooler(sequence_output)

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-145935> in <module> 
 18 
 19 embedding_output = teacher . bert . embeddings ( [ input_ids , position_ids , token_type_ids ] , training = training ) 
 ---> 20 encoder_outputs = teacher . bert . encoder . layer [ 0 ] ( [ embedding_output , extended_attention_mask , head_mask ] , training = training ) 
 21 
 22 sequence_output = encoder_outputs [ 0 ] 

 /databricks/python/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py in __call__ (self, inputs, *args, **kwargs) 
 889 with base_layer_utils.autocast_context_manager(
 890 self._compute_dtype):
 --> 891 outputs = self . call ( cast_inputs , * args , ** kwargs ) 
 892 self . _handle_activity_regularization ( inputs , outputs ) 
 893 self . _set_mask_metadata ( inputs , outputs , input_masks ) 

 /databricks/python/lib/python3.7/site-packages/transformers/modeling_tf_bert.py in call (self, inputs, training) 
 329 hidden_states , attention_mask , head_mask = inputs
 330 
 --> 331 attention_outputs = self . attention ( [ hidden_states , attention_mask , head_mask ] , training = training ) 
 332 attention_output = attention_outputs [ 0 ] 
 333 intermediate_output = self . intermediate ( attention_output ) 

 /databricks/python/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py in __call__ (self, inputs, *args, **kwargs) 
 889 with base_layer_utils.autocast_context_manager(
 890 self._compute_dtype):
 --> 891 outputs = self . call ( cast_inputs , * args , ** kwargs ) 
 892 self . _handle_activity_regularization ( inputs , outputs ) 
 893 self . _set_mask_metadata ( inputs , outputs , input_masks ) 

 /databricks/python/lib/python3.7/site-packages/transformers/modeling_tf_bert.py in call (self, inputs, training) 
 278 input_tensor , attention_mask , head_mask = inputs
 279 
 --> 280 self_outputs = self . self_attention ( [ input_tensor , attention_mask , head_mask ] , training = training ) 
 281 attention_output = self . dense_output ( [ self_outputs [ 0 ] , input_tensor ] , training = training ) 
 282 outputs = ( attention_output , ) + self_outputs [ 1 : ] # add attentions if we output them 

 /databricks/python/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py in __call__ (self, inputs, *args, **kwargs) 
 889 with base_layer_utils.autocast_context_manager(
 890 self._compute_dtype):
 --> 891 outputs = self . call ( cast_inputs , * args , ** kwargs ) 
 892 self . _handle_activity_regularization ( inputs , outputs ) 
 893 self . _set_mask_metadata ( inputs , outputs , input_masks ) 

 /databricks/python/lib/python3.7/site-packages/transformers/modeling_tf_bert.py in call (self, inputs, training) 
 236 # Mask heads if we want to 
 237 if head_mask is not None : 
 --> 238 attention_probs = attention_probs * head_mask
 239 
 240 context_layer = tf . matmul ( attention_probs , value_layer ) 

 /databricks/python/lib/python3.7/site-packages/tensorflow_core/python/ops/math_ops.py in binary_op_wrapper (x, y) 
 901 try : 
 902 y = ops.convert_to_tensor_v2(
 --> 903 y, dtype_hint=x.dtype.base_dtype, name="y")
 904 except TypeError : 
 905 # If the RHS is not a tensor, it might be a tensor aware object 

 /databricks/python/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py in convert_to_tensor_v2 (value, dtype, dtype_hint, name) 
 1240 name = name , 
 1241 preferred_dtype = dtype_hint , 
 -> 1242 as_ref=False)
 1243 
 1244 

 /databricks/python/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py in internal_convert_to_tensor (value, dtype, name, as_ref, preferred_dtype, ctx, accept_composite_tensors) 
 1294 
 1295 if ret is None : 
 -> 1296 ret = conversion_func ( value , dtype = dtype , name = name , as_ref = as_ref ) 
 1297 
 1298 if ret is NotImplemented : 

 /databricks/python/lib/python3.7/site-packages/tensorflow_core/python/framework/constant_o

In [15]:
%sh
ls /tmp/experiment3/teacher

config.json
tf_model.h5

In [16]:
# layer_num = 0
# print("the {}th encoder layer has {} variables and {} trainable variables".format(
#   layer_num,
#   len(teacher.layers[0].encoder.layer[layer_num].variables),
#   len(teacher.layers[0].encoder.layer[layer_num].trainable_variables)
# ))

# print("the inputs are transformed by embeddings layer first")
# for v in teacher.bert.embeddings.variables:
#   print(v.name)
  
# print("each encoder layer has attention, intermediate and bert_output layer")
# for v in teacher.bert.encoder.layer[layer_num].attention.variables:
#   print(v.name)
  
# for v in teacher.bert.encoder.layer[layer_num].intermediate.variables:
#   print(v.name)

# for v in teacher.bert.encoder.layer[layer_num].bert_output.variables:
#   print(v.name)

# print("the output of the encoder is a sequence output then is transformed into a pooled output through a pooler layer on top")
# for v in teacher.bert.pooler.variables:
#   print(v.name)


the 0th encoder layer has 16 variables and 16 trainable variables
each encoder layer has attention, intermediate and bert_output layer
tf_bert_for_sequence_classification/bert/encoder/layer_._0/attention/self/query/kernel:0
tf_bert_for_sequence_classification/bert/encoder/layer_._0/attention/self/query/bias:0
tf_bert_for_sequence_classification/bert/encoder/layer_._0/attention/self/key/kernel:0
tf_bert_for_sequence_classification/bert/encoder/layer_._0/attention/self/key/bias:0
tf_bert_for_sequence_classification/bert/encoder/layer_._0/attention/self/value/kernel:0
tf_bert_for_sequence_classification/bert/encoder/layer_._0/attention/self/value/bias:0
tf_bert_for_sequence_classification/bert/encoder/layer_._0/attention/output/dense/kernel:0
tf_bert_for_sequence_classification/bert/encoder/layer_._0/attention/output/dense/bias:0
tf_bert_for_sequence_classification/bert/encoder/layer_._0/attention/output/LayerNorm/gamma:0
tf_bert_for_sequence_classification/bert/encoder/layer_._0/attention/output/LayerNorm/beta:0
tf_bert_for_sequence_classification/bert/encoder/layer_._0/intermediate/dense/kernel:0
tf_bert_for_sequence_classification/bert/encoder/layer_._0/intermediate/dense/bias:0
tf_bert_for_sequence_classification/bert/encoder/layer_._0/output/dense/kernel:0
tf_bert_for_sequence_classification/bert/encoder/layer_._0/output/dense/bias:0
tf_bert_for_sequence_classification/bert/encoder/layer_._0/output/LayerNorm/gamma:0
tf_bert_for_sequence_classification/bert/encoder/layer_._0/output/LayerNorm/beta:0
tf_bert_for_sequence_classification/bert/pooler/dense/kernel:0
tf_bert_for_sequence_classification/bert/pooler/dense/bias:0

In [17]:
def extract_weights_from_teacher(teacher, student, layer_maps):
  """Extract the weights from teacher to initialize the student layer
  
  Args:
    teacher: BertSequenceClassifier
    student: BertSequenceClassifier
    layer_maps: e.g., {s: t} will initialize s'th student
                encoder layer using the weights from t'th teacher encoder layer
  """
  # copy the embedding weights
  for v in teacher.bert.embeddings.variables:
    student.bert.embeddings.add_weight(
      name=v.name,
      shape=v.shape,
      dtype=v.dtype,
      trainable=True,
      getter=lambda *_, **__: v
    )

  # initialize the student layer (s_layer_num) using the teacher layer number (layer_num)
  layer_num = 0
  s_layer_num = 0
  for s, t in layer_maps.items():
    # copy the encoder weights

    for v in teacher.bert.encoder.layer[t].attention.variables:
      student.bert.encoder.layer[s].add_weight(
        name=v.name,
        shape=v.shape,
        dtype=v.dtype,
        trainable=True,
        getter=lambda *_, **__: v
      )

    for v in teacher.bert.encoder.layer[t].intermediate.variables:
      student.bert.encoder.layer[s].add_weight(
        name = v.name, 
        shape = v.shape,
        dtype=v.dtype,
        trainable=True,
        getter=lambda *_, **__: v
      )

    for v in teacher.bert.encoder.layer[t].bert_output.variables:
      student.bert.encoder.layer[s].add_weight(
        name=v.name,
        shape=v.shape,
        dtype=v.dtype,
        trainable=True,
        getter=lambda *_, **__: v
      )

  # copy the pooler weights
  for v in teacher.bert.pooler.variables:
    student.bert.pooler.add_weight(
      name=v.name,
      shape=v.shape,
      dtype=v.dtype,
      trainable=True,
      getter=lambda *_, **__: v
    )
    
#   # Forward the callable's regularization losses (if any).
#   if hasattr(teacher, "regularization_losses"):
#     for l in self._func.regularization_losses:
#       if not callable(l):
#         raise ValueError(
#             "hub.KerasLayer(obj) expects obj.regularization_losses to be an "
#             "iterable of callables, each returning a scalar loss term.")
#       self.add_loss(self._call_loss_if_trainable(l))  # Supports callables.

In [18]:
# input_ids = tf.keras.layers.Input(shape=(config.max_seq_len,), dtype=tf.int32, name="input_ids")
# student_logits = student(input_ids)
# student_model = tf.keras.Model(inputs=input_ids,
#                        outputs=student_logits)

# teacher_input_ids = tf.keras.layers.Input(shape=(config.max_seq_len,), dtype=tf.int32, name="teacher_input_ids")
# teacher_logits = teacher(teacher_input_ids)
# teacher_model = tf.keras.Model(inputs=teacher_input_ids,
#                                outputs=teacher_logits)

In [19]:
teacher.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type) Output Shape Param # 
=================================================================
bert (TFBertMainLayer) multiple 108310272 
_________________________________________________________________
dropout_37 (Dropout) multiple 0 
_________________________________________________________________
classifier (Dense) multiple 1538 
=================================================================
Total params: 108,311,810
Trainable params: 108,311,810
Non-trainable params: 0
_________________________________________________________________

In [20]:
student.summary()

Model: "tf_bert_for_sequence_classification_20"
_________________________________________________________________
Layer (type) Output Shape Param # 
=================================================================
bert (TFBertMainLayer) multiple 31515648 
_________________________________________________________________
dropout_510 (Dropout) multiple 0 
_________________________________________________________________
classifier (Dense) multiple 1538 
=================================================================
Total params: 31,517,186
Trainable params: 31,517,186
Non-trainable params: 0
_________________________________________________________________

Prepare Data from MRPC

In [22]:
data = tensorflow_datasets.load('glue/mrpc')

INFO:absl:Load pre-computed datasetinfo (eg: splits) from bucket.
INFO:absl:Loading info from GCS for glue/mrpc/0.0.2
INFO:absl:Generating dataset glue (/root/tensorflow_datasets/glue/mrpc/0.0.2)
 Downloading and preparing dataset glue (1.43 MiB) to /root/tensorflow_datasets/glue/mrpc/0.0.2... 
Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]INFO:absl:Downloading https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2Fmrpc_dev_ids.tsv?alt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc into /root/tensorflow_datasets/downloads/fire.goog.com_v0_b_mtl-sent-repr.apps.com_o_2FjSIMlCiqs1QSmIykr4IRPnEHjPuGwAz5i40v8K9U0Z8.tsvalt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc.tmp.fa5987007dbe4ad0a52a1a195022d3c6...
Dl Completed...: 0%| | 0/1 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]INFO:absl:Downloading https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt into /root/tensorflow_datasets/downloads/dl.fbaip.com_sente_sente_msr_parap_trainfGxPZuQWGBti4Tbd1YNOwQr-OqxPejJ7gcp0Al6mlSk.txt.tmp.35ed895be7954d0abd5d23359aea31a1...
Dl Completed...: 0%| | 0/2 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]INFO:absl:Downloading https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_test.txt into /root/tensorflow_datasets/downloads/dl.fbaip.com_sente_sente_msr_parap_test0PdekMcyqYR-w4Rx_d7OTryq0J3RlYRn4rAMajy9Mak.txt.tmp.2e1be8f75f9f44aca1ba63f37f1a52c7...
Dl Completed...: 0%| | 0/3 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]/databricks/python/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
 InsecureRequestWarning)
/databricks/python/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
 InsecureRequestWarning)
/databricks/python/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
 InsecureRequestWarning)
Dl Completed...: 0%| | 0/3 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]Dl Completed...: 0%| | 0/3 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]Dl Completed...: 0%| | 0/3 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]Dl Completed...: 33%|███▎ | 1/3 [00:00<00:00, 2.55 url/s]Dl Completed...: 33%|███▎ | 1/3 [00:00<00:00, 2.55 url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]Dl Completed...: 67%|██████▋ | 2/3 [00:00<00:00, 2.55 url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]Dl Completed...: 100%|██████████| 3/3 [00:00<00:00, 2.55 url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 100%|██████████| 3/3 [00:00<00:00, 6.46 url/s]
INFO:absl:Generating split train
INFO:absl:Writing TFRecords
0 examples [00:00, ? examples/s]250 examples [00:00, 2497.87 examples/s]510 examples [00:00, 2525.47 examples/s]774 examples [00:00, 2556.17 examples/s]1023 examples [00:00, 2533.54 examples/s]1286 examples [00:00, 2559.99 examples/s]1549 examples [00:00, 2579.22 examples/s]1819 examples [00:00, 2612.82 examples/s]2088 examples [00:00, 2635.16 examples/s]2343 examples [00:00, 2608.13 examples/s]2609 examples [00:01, 2622.08 examples/s]2879 examples [00:01, 2643.93 examples/s]3140 examples [00:01, 2633.05 examples/s]3400 examples [00:01, 2606.93 examples/s]3659 examples [00:01, 2589.45 examples/s] Shuffling...: 0%| | 0/1 [00:00<?, ? shard/s]WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow_datasets/core/file_format_adapter.py:209: tf_record_iterator (f

In [23]:
train_dataset = glue_convert_examples_to_features(data['train'], teacher_tokenizer, max_length=config.max_seq_len, task='mrpc')
valid_dataset = glue_convert_examples_to_features(data['validation'], teacher_tokenizer, max_length=config.max_seq_len, task='mrpc')
train_dataset = train_dataset.shuffle(100).batch(16).repeat(2)
valid_dataset = valid_dataset.batch(32)

Training the student model ONLY

Training the knowledge distiller

In [26]:
# loss1 = ce_loss_fct_torch(torch.tensor([1.0,1.0]), torch.tensor([0.0, 0.0]), config)
# loss2 = ce_loss_fct(tf.constant([1.0,1.0]), tf.constant([0.0,0.0]), config)
# assert(loss1.numpy()==loss2.numpy())

loss1 = mse_loss_fct_torch(torch.tensor([[1.0,1.0],[2.0,2.0]]), torch.tensor([[0.0,0.0],[1.0, 1.0]]))
loss2 = mse_loss_fct(tf.constant([[1.0,1.0],[2.0,2.0]]), tf.constant([[0.0,0.0],[1.0, 1.0]]))
assert(loss1.numpy()==loss2.numpy())

In [27]:
import torch.nn.functional as F
import torch

def loss_fn_v2(teacher_logits, student_logits, targets, config):
  loss_op_standard = standard_loss_fct(student_logits, targets, config)
  mse_loss = mse_loss_fct(teacher_logits, student_logits)
  return loss_op_standard + mse_loss

def loss_fn_v1(teacher_logits, student_logits, targets, config):
  loss_op_standard = standard_loss_fct(student_logits, targets, config)
  loss_op_soft = ce_loss_fct(student_logits, teacher_logits, config)
  return loss_op_standard + loss_op_soft

def standard_loss_fct(student_logits, targets, config):
  one_hot_targets = tf.one_hot(targets, config.num_classes, dtype=tf.float32)
  loss_op_standard = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=student_logits, labels=one_hot_targets
  ))
  return loss_op_standard

def ce_loss_fct(student_logits, teacher_logits, config):
  teacher_targets = tf.nn.softmax(tf.multiply(teacher_logits,  1.0 / config.distill_temperature))
  loss_op_soft = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=tf.multiply(student_logits, 1.0 / config.distill_temperature), labels=tf.stop_gradient(teacher_targets)
  ))
  # scale soft target obj to match hard target obj. scale
  loss_op_soft *= tf.square(config.distill_temperature)
  return loss_op_soft

def ce_loss_fct_torch(student_logits, teacher_logits, config):
  loss = torch.nn.KLDivLoss(reduction='batchmean')
  return loss(F.log_softmax(student_logits/config.distill_temperature, dim=-1),
              F.softmax(teacher_logits/config.distill_temperature, dim=-1))*(config.distill_temperature)**2

def mse_loss_fct(teacher_logits, student_logits):
  return tf.reduce_mean(tf.keras.losses.MSE(teacher_logits, student_logits))

def mse_loss_fct_torch(teacher_logits, student_logits):
  return torch.nn.MSELoss(reduction='mean')(teacher_logits, student_logits)

In [28]:
def get_loss_acc(student, teacher, valid_dataset, config):
  mean_loss = tf.keras.metrics.Mean()
  acc = tf.keras.metrics.SparseCategoricalAccuracy()
  for x, y in valid_dataset:
    student_logits = student(x)[0]
    teacher_logits = teacher(x)[0]
    pred = tf.nn.softmax(student_logits)
    loss = loss_fn_v1(teacher_logits, student_logits, y, config)
    acc(y, pred)
    mean_loss(loss)
  return mean_loss.result().numpy(), acc.result().numpy()

In [29]:
# optimizer = tf.keras.optimizers.Adam(learning_rate=config.learning_rate, epsilon=config.adam_epsilon, clipnorm=config.max_grad_norm)
# ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, net=student)
# manager = tf.train.CheckpointManager(ckpt, '/tmp/distil', max_to_keep=5)
# ckpt = tf.train.Checkpoint(step=tf.Variable(1), opt=optimizer, net=student)
# ckpt.restore('/dbfs/ml/judith/transformers/ckpts/ckpt-2').assert_consumed()

In [30]:
%fs
ls dbfs:/ml/judith/transformers/distilbert/experiment3

path,name,size
dbfs:/ml/judith/transformers/distilbert/experiment3/config.json,config.json,188


In [31]:
import os

# setup directory
save_dir = "/dbfs/ml/judith/transformers/distilbert/experiment3"
local_dir = "/tmp/experiment3/"
if not os.path.exists(local_dir):
  os.mkdir(local_dir)

if not os.path.exists(save_dir):
  os.mkdir(save_dir)

# optimizer
config.learning_rate = 5e-4
config.save_pretrained(save_dir)
optimizer = tf.keras.optimizers.Adam(learning_rate=config.learning_rate, epsilon=config.adam_epsilon, clipnorm=config.max_grad_norm)

In [32]:
train_loss = []
train_acc = []

for epoch in range(config.epoch):
  epoch_end_loss = tf.keras.metrics.Mean()
  epoch_end_acc = tf.keras.metrics.SparseCategoricalAccuracy()
  
  # Training loop - using batches of 16
  for step, (x, y) in enumerate(train_dataset):
    with tf.GradientTape() as tape:
      student_logits = student(x)[0]
      teacher_logits = teacher(x)[0]
      targets = y
      loss = loss_fn_v1(teacher_logits, student_logits, targets, config)
    
    # only optimize student weights
    gradients = tape.gradient(loss, student.trainable_variables)
    optimizer.apply_gradients(zip(gradients, student.trainable_variables))

    # logging
    if step % 100 == 0:
      print(step, float(loss))
    
    # log accuracy
    pred = tf.nn.softmax(student_logits)
    epoch_end_acc(targets, pred)
    epoch_end_loss(loss)
  
  # End epoch
  train_acc.append(epoch_end_acc.result())
  train_loss.append(epoch_end_loss.result())
  
  val_loss, val_acc = get_loss_acc(student, teacher, valid_dataset, config)
  print("epoch : {} training loss: {} training acc: {}".format(epoch, epoch_end_loss.result(), epoch_end_acc.result()))
  print("epoch : {} validation loss: {} validation acc: {}".format(epoch, val_loss, val_acc))
  
  # save checkpoint
#   ckpt.step.assign_add(1)
#   save_path = manager.save()
#   print("Saved checkpoint for epoch {} at {}".format(int(ckpt.step), save_path))
#   print("epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch, epoch_end_loss.result(), epoch_end_acc.result()))

  # save model weights
  student.save_weights(os.path.join(local_dir,"ckpt-{}".format(epoch)), save_format='tf')

In [33]:
get_loss_acc(student, teacher, valid_dataset, config)

Out[26]: (3.4394732, 0.6838235)

In [34]:
for epoch in range(config.epoch):
  student.load_weights("/dbfs/ml/judith/transformers/ckpts/{}".format(epoch))
  loss, acc = get_loss_acc(student, teacher, train_dataset, config)
  print("epoch : {} training loss: {} training acc: {}".format(epoch, loss, acc))

epoch : 0 training loss: 3.4508979320526123 training acc: 0.674481987953186
epoch : 1 training loss: 3.446786642074585 training acc: 0.674481987953186
epoch : 2 training loss: 3.4454026222229004 training acc: 0.674481987953186
epoch : 3 training loss: 3.441863775253296 training acc: 0.674481987953186
epoch : 4 training loss: 3.4411721229553223 training acc: 0.674481987953186

In [35]:
for epoch in range(config.epoch):
  student.load_weights("/dbfs/ml/judith/transformers/ckpts/{}".format(epoch))
  loss, acc = get_loss_acc(student, teacher, valid_dataset, config)
  print("epoch : {} validation loss: {} validation acc: {}".format(epoch, loss, acc))

epoch : 0 validation loss: 3.4499192237854004 validation acc: 0.6838235259056091
epoch : 1 validation loss: 3.4454843997955322 validation acc: 0.6838235259056091
epoch : 2 validation loss: 3.4440395832061768 validation acc: 0.6838235259056091
epoch : 3 validation loss: 3.439988613128662 validation acc: 0.6838235259056091
epoch : 4 validation loss: 3.4394731521606445 validation acc: 0.6838235259056091

In [36]:
for epoch in range(config.epoch):
  student.load_weights("/dbfs/ml/judith/transformers/ckpts/{}".format(epoch))
  train_acc = get_valid_acc(student, train_dataset)
  acc = get_valid_acc(student, valid_dataset)
  print("epoch : {} train acc: {} validation acc: {}".format(epoch, train_acc.numpy(), acc.numpy()))

epoch : 0 train acc: 0.674481987953186 validation acc: 0.6838235259056091
epoch : 1 train acc: 0.674481987953186 validation acc: 0.6838235259056091
epoch : 2 train acc: 0.674481987953186 validation acc: 0.6838235259056091
epoch : 3 train acc: 0.674481987953186 validation acc: 0.6838235259056091
epoch : 4 train acc: 0.674481987953186 validation acc: 0.6838235259056091

In [37]:
res = get_valid_acc(student, valid_dataset)
res.numpy()

Out[18]: 0.6838235

In [38]:
%fs
cp -r file:/tmp/distilbert dbfs:/ml/judith/transformers/ckpts

res0: Boolean = true

In [39]:
tf.argmax(results,1)

Out[54]: <tf.Tensor: id=2586014, shape=(24,), dtype=int64, numpy=
array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1])>

In [40]:
tf.equal(tf.argmax(results, 1), tf.argmax())

Out[46]: array([[-0.60986143, 1.0837568 ],
 [ 0.09118471, 0.29516476],
 [-0.7158415 , 1.1913157 ],
 [-0.4526738 , 0.923979 ],
 [-0.19267556, 0.65840644],
 [-0.65269977, 1.1091357 ],
 [ 0.71593946, -0.49036485],
 [-0.62001866, 1.0765138 ],
 [-0.65391034, 1.1503623 ],
 [-0.2535582 , 0.72101456],
 [-0.42134604, 0.87415594],
 [-0.719673 , 1.2054204 ],
 [-0.6649697 , 1.1418422 ],
 [-0.37266234, 0.8350723 ],
 [-0.68396103, 1.181153 ],
 [-0.34383416, 0.80864 ],
 [-0.4569165 , 0.94713295],
 [-0.08806867, 0.53173596],
 [-0.7284867 , 1.2087333 ],
 [-0.12023775, 0.57294476],
 [-0.51208216, 0.97944826],
 [-0.06646109, 0.47441307],
 [-0.13662477, 0.5703864 ],
 [-0.4707438 , 0.9563202 ],
 [-0.54565 , 1.0553306 ],
 [-0.65720594, 1.1375569 ],
 [-0.603665 , 1.0736836 ],
 [-0.5706943 , 1.0548316 ],
 [-0.63253826, 1.1077539 ],
 [-0.17473526, 0.6080356 ],
 [-0.5258696 , 0.9823329 ],
 [-0.6367127 , 1.1030618 ],
 [-0.6464472 , 1.1340488 ],
 [-0.47155175, 0.9452964 ],
 [-0.47981796, 0.9446642 ],
 [-0.16580467, 0.62852514],
 [ 0.13576464, 0.26995686],
 [-0.3330386 , 0.79294926],
 [-0.67616814, 1.1361644 ],
 [-0.5728045 , 1.0284945 ],
 [-0.5234155 , 1.0313827 ],
 [-0.71388495, 1.1600287 ],
 [-0.67785823, 1.1390435 ],
 [-0.34962395, 0.80862415],
 [-0.2661931 , 0.7122902 ],
 [-0.5598904 , 1.0254445 ],
 [-0.6257291 , 1.1060236 ],
 [-0.47577593, 0.9198599 ],
 [-0.34090135, 0.79095286],
 [ 0.04019816, 0.3601394 ],
 [-0.17880383, 0.6337542 ],
 [-0.4719053 , 0.9582414 ],
 [-0.36412588, 0.81866926],
 [ 0.33677736, 0.00894536],
 [-0.6666045 , 1.1421952 ],
 [-0.68127775, 1.151847 ],
 [-0.5248226 , 0.97841525],
 [-0.61063206, 1.048671 ],
 [-0.70198333, 1.1634849 ],
 [ 0.18742082, 0.20412788],
 [-0.46214527, 0.9319813 ],
 [-0.10471676, 0.5197073 ],
 [-0.5259552 , 1.004858 ],
 [-0.6098042 , 1.0599213 ],
 [-0.4391064 , 0.8936264 ],
 [-0.1867569 , 0.59780186],
 [ 0.01940855, 0.38897607],
 [-0.6644772 , 1.1325732 ],
 [-0.69174916, 1.1506382 ],
 [ 0.13589337, 0.2673158 ],
 [-0.6700962 , 1.153359 ],
 [-0.27388388, 0.74286956],
 [-0.49037316, 0.9955026 ],
 [ 0.06454299, 0.32583514],
 [-0.6437716 , 1.1110007 ],
 [-0.64400685, 1.1352906 ],
 [-0.48827204, 0.94044757],
 [ 0.3493447 , -0.01391271],
 [-0.44950223, 0.9450929 ],
 [-0.44175732, 0.89276075],
 [-0.12980844, 0.58551854],
 [-0.56955487, 1.0211033 ],
 [-0.5057421 , 0.9676329 ],
 [-0.52656007, 1.0051538 ],
 [-0.75315326, 1.2196054 ],
 [-0.73955667, 1.1934521 ],
 [-0.46626502, 0.92870414],
 [-0.20918107, 0.6667271 ],
 [-0.6871907 , 1.1455532 ],
 [ 0.19710787, 0.18170668],
 [-0.53174716, 0.9748563 ],
 [-0.54306877, 1.0390803 ],
 [-0.00926425, 0.40115273],
 [-0.6642691 , 1.1283369 ],
 [-0.09411409, 0.53803265],
 [-0.3017078 , 0.75204587],
 [ 0.4829687 , -0.15538216],
 [ 0.09240162, 0.28422883],
 [-0.5689018 , 1.0553023 ],
 [ 0.03437992, 0.38740957],
 [-0.6761581 , 1.166359 ],
 [ 0.3739927 , -0.0250047 ],
 [-0.625628 , 1.1148757 ],
 [-0.40893525, 0.8901157 ],
 [-0.70678365, 1.1496271 ],
 [-0.5310745 , 0.9906875 ],
 [-0.7055363 , 1.1751262 ],
 [-0.7160511 , 1.2080854 ],
 [-0.47041994, 0.9091687 ],
 [ 0.3139905 , 0.05027173],
 [-0.19564812, 0.6145728 ],
 [-0.6886486 , 1.1997702 ],
 [ 0.09650204, 0.29227754],
 [ 0.15494311, 0.22039104],
 [-0.6474567 , 1.1244891 ],
 [-0.42002368, 0.88138753],
 [ 0.06390077, 0.31207672],
 [-0.6923533 , 1.1554077 ],
 [-0.40595224, 0.84243625],
 [-0.57191527, 1.0365535 ],
 [ 0.06131126, 0.37641776],
 [-0.5390514 , 1.0298933 ],
 [-0.75057185, 1.2168492 ],
 [-0.05449066, 0.45387274],
 [-0.51358753, 0.990801 ],
 [ 0.74454826, -0.52988833],
 [-0.17847995, 0.60256094],
 [-0.5359979 , 1.0136193 ],
 [-0.4672489 , 0.95812005],
 [-0.11980329, 0.5517218 ],
 [-0.17644118, 0.6146549 ],
 [-0.45917714, 0.9457082 ],
 [-0.40235808, 0.8993072 ],
 [-0.6301533 , 1.116043 ],
 [-0.07684531, 0.4793113 ],
 [-0.6808449 , 1.1424247 ],
 [-0.13641942, 0.5501804 ],
 [-0.45896402, 0.91506624],
 [-0.5837547 , 1.0716311 ],
 [-0.6697946 , 1.1628697 ],
 [-0.63827807, 1.0947073 ],
 [-0.6820466 , 1.1554333 ],
 [-0.7536713 , 1.207

In [41]:
input_words = ["dogs are on the ground", "cats are in the cloud"]
input_word_ids = tf.constant([teacher_tokenizer.encode(text) for text in input_words])
targets = tf.constant([1, 0])

In [42]:
teacher_logits = teacher(input_word_ids)
student_logits = student(input_word_ids)

In [43]:
loss_fn(teacher_logits, student_logits, targets, config)

Out[61]: <tf.Tensor: id=71154, shape=(), dtype=float32, numpy=0.6950183>

In [44]:
amax_seq_length = 128  # Your choice here.
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
attention_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="attention_mask")
token_type_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="token_type_ids")
bert_layer = hub.KerasLayer("/dbfs/ml/smrt-hub/bert-base-uncased/1",
                            trainable=True)
sequence_output, pooled_output = bert_layer([input_word_ids, attention_mask, token_type_ids])


In [45]:
dir(bert_layer.resolved_object)

Out[6]: ['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_variable_with_custom_getter',
 '_checkpoint_dependencies',
 '_default_save_signature',
 '_deferred_dependencies',
 '_gather_saveables_for_checkpoint',
 '_handle_deferred_dependencies',
 '_is_hub_module_v1',
 '_list_extra_dependencies_for_serialization',
 '_list_functions_for_serialization',
 '_lookup_dependency',
 '_maybe_initialize_trackable',
 '_name_based_attribute_restore',
 '_name_based_restores',
 '_no_dependency',
 '_object_identifier',
 '_preload_simple_restoration',
 '_restore_from_checkpoint_position',
 '_self_name_based_restores',
 '_self_setattr_tracking',
 '_self_unconditional_checkpoint_dependencies',
 '_self_unconditional_deferred_dependencies',
 '_self_unconditional_dependency_names',
 '_self_update_uid',
 '_setattr_tracking',
 '_single_restoration_from_checkpoint_position',
 '_track_trackable',
 '_tracking_metadata',
 '_unconditional_checkpoint_dependencies',
 '_unconditional_dependency_names',
 '_update_uid',
 'call_and_return_all_conditional_losses',
 'keras_api',
 'layer-0',
 'layer-1',
 'layer-2',
 'layer-3',
 'layer-4',
 'layer-5',
 'layer_with_weights-0',
 'regularization_losses',
 'signatures',
 'tensorflow_git_version',
 'tensorflow_version',
 'trainable_variables',
 'variables']

In [46]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()

In [47]:
# Prepare dataset for GLUE as a tf.data.Dataset instance

data = tensorflow_datasets.load('glue/mrpc')

# train_dataset = glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, task='mrpc')
# valid_dataset = glue_convert_examples_to_features(data['validation'], tokenizer, max_length=128, task='mrpc')
# train_dataset = train_dataset.shuffle(100).batch(32).repeat(2)
# valid_dataset = valid_dataset.batch(64)

INFO:absl:Overwrite dataset info from restored data version.
INFO:absl:Reusing dataset glue (/root/tensorflow_datasets/glue/mrpc/0.0.2)
INFO:absl:Constructing tf.data.Dataset for split None, from /root/tensorflow_datasets/glue/mrpc/0.0.2

Train DistillBert

In [49]:
student_config_class, student_model_class, student_tokenizer_class = DistilBertConfig, TFDistilBertForSequenceClassification, DistilBertTokenizer
teacher_config_class, teacher_model_class, teacher_tokenizer_class = BertConfig, TFBertForSequenceClassification, BertTokenizer

In [50]:
tokenizer = teacher_tokenizer_class.from_pretrained('bert-base-uncased')

0%| | 0/231508 [00:00<?, ?B/s] 15%|█▌ | 34816/231508 [00:00<00:00, 244937.24B/s] 75%|███████▌ | 174080/231508 [00:00<00:00, 321776.45B/s]100%|██████████| 231508/231508 [00:00<00:00, 888860.24B/s]

In [51]:
student_config = {
	"activation": "gelu",
	"attention_dropout": 0.1,
	"dim": 768,
	"dropout": 0.1,
	"hidden_dim": 3072,
	"initializer_range": 0.02,
	"max_position_embeddings": 512,
	"n_heads": 12,
	"n_layers": 6,
	"sinusoidal_pos_embds": true,
	"tie_weights_": true,
	"vocab_size": 30522
  }

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-145069> in <module> 
 9 "n_heads" : 12 , 
 10 "n_layers" : 6 , 
 ---> 11 "sinusoidal_pos_embds" : true , 
 12 "tie_weights_" : true , 
 13 "vocab_size" : 30522 

 NameError : name 'true' is not defined

In [52]:
# Student
stu_architecture_config = student_config_class.from_pretrained("/dbfs/ml/judith/transformers/distilbert-base-uncased.json")
stu_architecture_config.sinusoidal_pos_embds = False
student = student_model_class(stu_architecture_config)

In [53]:
teacher = teacher_model_class.from_pretrained("bert-base-uncased")#, output_hidden_states=True)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-145070> in <module> 
 ----> 1 teacher = teacher_model_class . from_pretrained ( "bert-base-uncased" ) #, output_hidden_states=True) 

 NameError : name 'teacher_model_class' is not defined

Evaluation using fine-tuned model

In [55]:
# Load dataset, tokenizer, model from pretrained model/vocabulary
tokenizer = DistilBertTokenizer.from_pretrained('/dbfs/ml/judith/transformers/mrpc/2')
model = TFDistilBertForSequenceClassification.from_pretrained('/dbfs/ml/judith/transformers/mrpc/2')

In [56]:
results = model.predict(valid_dataset)

In [57]:
gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

In [58]:
# Load dataset, tokenizer, model from pretrained model/vocabulary
teacher_tokenizer = BertTokenizer.from_pretrained('/dbfs/ml/judith/transformers/mrpc/1')
teacher_model = TFBertForSequenceClassification.from_pretrained('/dbfs/ml/judith/transformers/mrpc/1')

In [59]:
results = teacher_model.predict(valid_dataset)

Prepare Dataset from MRPC

In [61]:
data = tensorflow_datasets.load('glue/mrpc')

INFO:absl:Load pre-computed datasetinfo (eg: splits) from bucket.
INFO:absl:Loading info from GCS for glue/mrpc/0.0.2
INFO:absl:Generating dataset glue (/root/tensorflow_datasets/glue/mrpc/0.0.2)
 Downloading and preparing dataset glue (1.43 MiB) to /root/tensorflow_datasets/glue/mrpc/0.0.2... 
Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]INFO:absl:Downloading https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2Fmrpc_dev_ids.tsv?alt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc into /root/tensorflow_datasets/downloads/fire.goog.com_v0_b_mtl-sent-repr.apps.com_o_2FjSIMlCiqs1QSmIykr4IRPnEHjPuGwAz5i40v8K9U0Z8.tsvalt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc.tmp.24178278854045c4813278abb8592eec...
Dl Completed...: 0%| | 0/1 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]INFO:absl:Downloading https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt into /root/tensorflow_datasets/downloads/dl.fbaip.com_sente_sente_msr_parap_trainfGxPZuQWGBti4Tbd1YNOwQr-OqxPejJ7gcp0Al6mlSk.txt.tmp.09661eca187543a1842a1793db0336ef...
Dl Completed...: 0%| | 0/2 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]INFO:absl:Downloading https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_test.txt into /root/tensorflow_datasets/downloads/dl.fbaip.com_sente_sente_msr_parap_test0PdekMcyqYR-w4Rx_d7OTryq0J3RlYRn4rAMajy9Mak.txt.tmp.3ae4ea2992c74b1888bbb6c9ea6f40a9...
Dl Completed...: 0%| | 0/3 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]/databricks/python/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
 InsecureRequestWarning)
/databricks/python/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
 InsecureRequestWarning)
/databricks/python/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
 InsecureRequestWarning)
Dl Completed...: 0%| | 0/3 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]Dl Completed...: 0%| | 0/3 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]Dl Completed...: 33%|███▎ | 1/3 [00:00<00:00, 2.18 url/s]Dl Completed...: 33%|███▎ | 1/3 [00:00<00:00, 2.18 url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]Dl Completed...: 67%|██████▋ | 2/3 [00:00<00:00, 2.18 url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]Dl Completed...: 67%|██████▋ | 2/3 [00:00<00:00, 2.18 url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]Dl Completed...: 100%|██████████| 3/3 [00:00<00:00, 2.77 url/s]Dl Completed...: 100%|██████████| 3/3 [00:00<00:00, 2.77 url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]


INFO:absl:Generating split train
INFO:absl:Writing TFRecords
0 examples [00:00, ? examples/s]257 examples [00:00, 2568.45 examples/s]506 examples [00:00, 2544.35 examples/s]768 examples [00:00, 2565.91 examples/s]1029 examples [00:00, 2577.93 examples/s]1292 examples [00:00, 2591.47 examples/s]1556 examples [00:00, 2603.95 examples/s]1805 examples [00:00, 2566.62 examples/s]2066 examples [00:00, 2578.02 examples/s]2324 examples [00:00, 2576.49 examples/s]2584 examples [00:01, 2581.04 examples/s]2846 examples [00:01, 2592.02 examples/s]3101 examples [00:01, 2534.43 examples/s]3364 examples [00:01, 2560.35 examples/s]3637 examples [00:01, 2607.71 examples/s] Shuffling...: 0%| | 0/1 [00:00<?, ? shard/s]WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow_datasets/core/file_format_adapter.py:209: tf_rec

In [62]:
# Prepare dataset for GLUE as a tf.data.Dataset instance
train_dataset = glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, task='mrpc')
valid_dataset = glue_convert_examples_to_features(data['validation'], tokenizer, max_length=128, task='mrpc')
train_dataset = train_dataset.shuffle(100).batch(32).repeat(2)
valid_dataset = valid_dataset.batch(64)

DistillBert for MRPC

In [64]:
# Load dataset, tokenizer, model from pretrained model/vocabulary
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

0%| | 0/231508 [00:00<?, ?B/s] 15%|█▌ | 34816/231508 [00:00<00:00, 220780.88B/s] 97%|█████████▋| 225280/231508 [00:00<00:00, 292349.73B/s]100%|██████████| 231508/231508 [00:00<00:00, 727939.27B/s]
 0%| | 0/492 [00:00<?, ?B/s]100%|██████████| 492/492 [00:00<00:00, 417563.25B/s]
 0%| | 0/363423424 [00:00<?, ?B/s] 0%| | 52224/363423424 [00:00<17:58, 336936.35B/s] 0%| | 261120/363423424 [00:00<13:55, 434462.90B/s] 0%| | 940032/363423424 [00:00<10:03, 600316.42B/s] 1%| | 2228224/363423424 [00:00<07:10, 839122.67B/s] 1%|▏ | 4753408/363423424 [00:00<05:04, 1178015.86B/s] 2%|▏ | 7047168/363423424 [00:00<03:36, 1642903.68B/s] 3%|▎ | 9553920/363423424 [00:00<02:36, 2268326.95B/s] 3%|▎ | 11847680/363423424 [00:01<01:53, 3095521.11B/s] 4%|▍ | 14370816/363423424 [00:01<01:24, 4152717.39B/s] 5%|▍ | 16664576/363423424 [00:01<01:03, 5463848.69B/s] 5%|▌ | 19171328/363423424 [00:01<00:49, 6998458.41B/s] 6%|▌ | 21465088/363423424 [00:01<00:39, 8735442.93B/s] 7%|▋ | 23988224/363423424 [00:01<00:32, 10546369.10B/s] 7%|▋ | 26265600/363423424 [00:01<00:27, 12350809.82B/s] 8%|▊ | 28788736/363423424 [00:01<00:23, 14017607.25B/s] 9%|▊ | 31082496/363423424 [00:01<00:21, 15522485.79B/s] 9%|▉ | 33589248/363423424 [00:02<00:19, 16713664.95B/s] 10%|▉ | 35883008/363423424 [00:02<00:18, 17736393.18B/s] 11%|█ | 38406144/363423424 [00:02<00:17, 18483475.91B/s] 11%|█ | 40683520/363423424 [00:02<00:16, 19057897.65B/s] 12%|█▏ | 43206656/363423424 [00:02<00:16, 19460571.69B/s] 13%|█▎ | 45500416/363423424 [00:02<00:16, 19801888.89B/s] 13%|█▎ | 48023552/363423424 [00:02<00:15, 20009298.13B/s] 14%|█▍ | 50300928/363423424 [00:02<00:15, 20164280.69B/s] 15%|█▍ | 52824064/363423424 [00:03<00:15, 20260334.47B/s] 15%|█▌ | 55101440/363423424 [00:03<00:15, 20346319.38B/s] 16%|█▌ | 57624576/363423424 [00:03<00:15, 20377151.79B/s] 16%|█▋ | 59918336/363423424 [00:03<00:14, 20492481.26B/s] 17%|█▋ | 62425088/363423424 [00:03<00:14, 20440909.39B/s] 18%|█▊ | 64718848/363423424 [00:03<00:14, 20516868.83B/s] 19%|█▊ | 67241984/363423424 [00:03<00:14, 20497004.12B/s] 19%|█▉ | 69535744/363423424 [00:03<00:14, 20571137.39B/s] 20%|█▉ | 72042496/363423424 [00:03<00:14, 20496959.05B/s] 20%|██ | 74094592/363423424 [00:04<00:16, 17249508.55B/s] 21%|██ | 76269568/363423424 [00:04<00:17, 16272672.87B/s] 22%|██▏ | 78702592/363423424 [00:04<00:15, 18067320.87B/s] 22%|██▏ | 80623616/363423424 [00:04<00:17, 16416408.91B/s] 23%|██▎ | 83249152/363423424 [00:04<00:16, 16787156.87B/s] 24%|██▎ | 85994496/363423424 [00:04<00:14, 19001206.78B/s] 24%|██▍ | 88048640/363423424 [00:04<00:15, 17477132.78B/s] 25%|██▍ | 90687488/363423424 [00:05<00:15, 17494744.88B/s] 26%|██▌ | 93325312/363423424 [00:05<00:13, 19460782.75B/s] 26%|██▋ | 95406080/363423424 [00:05<00:15, 17620233.75B/s] 27%|██▋ | 97716224/363423424 [00:05<00:15, 17038840.59B/s] 28%|██▊ | 100611072/363423424 [00:05<00:13, 19437611.06B/s] 28%|██▊ | 102737920/363423424 [00:05<00:14, 17788214.33B/s] 29%|██▉ | 104925184/363423424 [00:05<00:15, 16808490.08B/s] 30%|██▉ | 107448320/363423424 [00:05<00:13, 18678901.05B/s] 30%|███ | 109465600/363423424 [00:06<00:14, 16968328.78B/s] 31%|███ | 111986688/363423424 [00:06<00:14, 17043446.53B/s] 32%|███▏ | 114722816/363423424 [00:06<00:14, 17195629.44B/s] 32%|███▏ | 117147648/363423424 [00:06<00:13, 18838692.73B/s] 33%|███▎ | 119126016/363423424 [00:06<00:14, 17185419.07B/s] 33%|███▎ | 120938496/363423424 [00:06<00:14, 16726500.91B/s] 34%|███▍ | 122680320/363423424 [00:06<00:15, 15226202.63B/s] 34%|███▍ | 124749824/363423424 [00:06<00:15, 15182215.24B/s] 35%|███▌ | 127390720/363423424 [00:07<00:13, 17401365.39B/s] 36%|███▌ | 129282048/363423424 [00:07<00:14, 16033004.13B/s] 36%|███▋ | 131958784/363423424 [00:07<00:13, 17471932.77B/s] 37%|███▋ | 133833728/363423424 [00:07<00:13, 17206767.56B/s] 38%|███▊ | 136513536/363423424 [00:07<00:12, 18425655.47B/s] 38%|███▊ | 138443776/363423424 [00:07<00:12, 18285861.94B/s] 39%|███▊ | 140626944/363423424 [0

In [65]:
%fs
mkdirs dbfs:/ml/judith/transformers/mrpc/2

res21: Boolean = true

In [66]:
%sh
mkdir /tmp/mrpc/2

In [67]:
# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule 
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Train and evaluate using tf.keras.Model.fit()
history = model.fit(train_dataset, epochs=2, steps_per_epoch=115,
                    validation_data=valid_dataset, validation_steps=7)


--------------------------------------------------------------------------- 
 AssertionError Traceback (most recent call last)
 <command-145002> in <module> 
 10 
 11 # Save model 
 ---> 12 model . save_pretrained ( "/tmp/mrpc/2/" ) 
 13 tokenizer . save_pretrained ( "/tmp/mrpc/2/" ) 

 /databricks/python/lib/python3.7/site-packages/transformers/modeling_tf_utils.py in save_pretrained (self, save_directory) 
 128 can be re - loaded using the ` : func : ` ~ transformers . PreTrainedModel . from_pretrained ` ` class method . 
 129 """
 --> 130 assert os . path . isdir ( save_directory ) , "Saving path should be a directory where the model and configuration can be saved" 
 131 
 132 # Save configuration file 

 AssertionError : Saving path should be a directory where the model and configuration can be saved

In [68]:
# Save model
model.save_pretrained("/tmp/mrpc/2/")
tokenizer.save_pretrained("/tmp/mrpc/2/")

Out[12]: ('/tmp/mrpc/2/vocab.txt',
 '/tmp/mrpc/2/special_tokens_map.json',
 '/tmp/mrpc/2/added_tokens.json')

In [69]:
%fs
ls dbfs:/ml/judith/transformers/mrpc/1

path,name,size
dbfs:/ml/judith/transformers/mrpc/1/added_tokens.json,added_tokens.json,2
dbfs:/ml/judith/transformers/mrpc/1/config.json,config.json,543
dbfs:/ml/judith/transformers/mrpc/1/special_tokens_map.json,special_tokens_map.json,112
dbfs:/ml/judith/transformers/mrpc/1/tf_model.h5,tf_model.h5,433518744
dbfs:/ml/judith/transformers/mrpc/1/tokenizer_config.json,tokenizer_config.json,59
dbfs:/ml/judith/transformers/mrpc/1/vocab.txt,vocab.txt,213450


In [70]:
%fs
cp -r file:/tmp/mrpc/2 dbfs:/ml/judith/transformers/mrpc/2

res1: Boolean = true

In [71]:
# Load dataset, tokenizer, model from pretrained model/vocabulary
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-cased')

0%| | 0/213450 [00:00<?, ?B/s] 24%|██▍ | 52224/213450 [00:00<00:00, 369507.54B/s]100%|██████████| 213450/213450 [00:00<00:00, 990101.12B/s]
 0%| | 0/313 [00:00<?, ?B/s]100%|██████████| 313/313 [00:00<00:00, 241459.84B/s]
 0%| | 0/526681800 [00:00<?, ?B/s] 0%| | 52224/526681800 [00:00<25:01, 350745.44B/s] 0%| | 295936/526681800 [00:00<19:07, 458578.02B/s] 0%| | 1253376/526681800 [00:00<13:46, 635483.17B/s] 1%| | 3804160/526681800 [00:00<09:42, 898242.44B/s] 1%| | 5213184/526681800 [00:00<07:00, 1240794.28B/s] 2%|▏ | 7982080/526681800 [00:00<05:01, 1722927.78B/s] 2%|▏ | 10927104/526681800 [00:00<03:34, 2401116.75B/s] 2%|▏ | 12733440/526681800 [00:01<02:40, 3201977.64B/s] 3%|▎ | 15616000/526681800 [00:01<01:57, 4366352.51B/s] 3%|▎ | 17643520/526681800 [00:01<01:31, 5575892.69B/s] 4%|▍ | 20433920/526681800 [00:01<01:09, 7336454.69B/s] 4%|▍ | 22594560/526681800 [00:01<00:57, 8817294.44B/s] 5%|▍ | 25267200/526681800 [00:01<00:46, 10832505.75B/s] 5%|▌ | 27372544/526681800 [00:01<00:40, 12448727.89B/s] 6%|▌ | 30018560/526681800 [00:01<00:34, 14443380.82B/s] 6%|▌ | 32143360/526681800 [00:01<00:32, 15009489.62B/s] 7%|▋ | 34786304/526681800 [00:02<00:28, 17244007.36B/s] 7%|▋ | 36967424/526681800 [00:02<00:28, 17237080.88B/s] 8%|▊ | 39619584/526681800 [00:02<00:25, 19259669.09B/s] 8%|▊ | 41845760/526681800 [00:02<00:25, 18839163.54B/s] 8%|▊ | 44312576/526681800 [00:02<00:23, 20276288.25B/s] 9%|▉ | 46520320/526681800 [00:02<00:24, 19462739.76B/s] 9%|▉ | 48942080/526681800 [00:02<00:23, 20676514.25B/s] 10%|▉ | 51124224/526681800 [00:02<00:24, 19592741.02B/s] 10%|█ | 53695488/526681800 [00:02<00:22, 21098403.78B/s] 11%|█ | 55902208/526681800 [00:03<00:23, 19924126.48B/s] 11%|█ | 58464256/526681800 [00:03<00:21, 21347033.26B/s] 12%|█▏ | 60684288/526681800 [00:03<00:23, 20080018.33B/s] 12%|█▏ | 63251456/526681800 [00:03<00:21, 21483357.80B/s] 12%|█▏ | 65480704/526681800 [00:03<00:22, 20155345.98B/s] 13%|█▎ | 67988480/526681800 [00:03<00:21, 21415444.11B/s] 13%|█▎ | 70203392/526681800 [00:03<00:22, 20166458.31B/s] 14%|█▍ | 72742912/526681800 [00:03<00:21, 21492729.90B/s] 14%|█▍ | 74962944/526681800 [00:04<00:22, 20277826.94B/s] 15%|█▍ | 77499392/526681800 [00:04<00:20, 21573625.55B/s] 15%|█▌ | 79724544/526681800 [00:04<00:22, 20298732.40B/s] 16%|█▌ | 82247680/526681800 [00:04<00:20, 21562903.69B/s] 16%|█▌ | 84469760/526681800 [00:04<00:21, 20211684.44B/s] 16%|█▋ | 86810624/526681800 [00:04<00:20, 21074763.48B/s] 17%|█▋ | 88972288/526681800 [00:04<00:21, 20055464.12B/s] 17%|█▋ | 91286528/526681800 [00:04<00:20, 20890733.78B/s] 18%|█▊ | 93572096/526681800 [00:04<00:20, 20802772.37B/s] 18%|█▊ | 95682560/526681800 [00:04<00:20, 20556419.90B/s] 19%|█▊ | 98104320/526681800 [00:05<00:19, 21532736.55B/s] 19%|█▉ | 100285440/526681800 [00:05<00:21, 20023470.46B/s] 20%|█▉ | 102861824/526681800 [00:05<00:19, 21451726.57B/s] 20%|█▉ | 105064448/526681800 [00:05<00:20, 20087779.61B/s] 20%|██ | 107637760/526681800 [00:05<00:19, 21501197.67B/s] 21%|██ | 109856768/526681800 [00:05<00:20, 20129745.61B/s] 21%|██▏ | 112487424/526681800 [00:05<00:19, 21654905.10B/s] 22%|██▏ | 114732032/526681800 [00:05<00:20, 20279350.35B/s] 22%|██▏ | 117149696/526681800 [00:05<00:19, 21309409.52B/s] 23%|██▎ | 119346176/526681800 [00:06<00:20, 20176793.41B/s] 23%|██▎ | 121780224/526681800 [00:06<00:19, 21268176.77B/s] 24%|██▎ | 123963392/526681800 [00:06<00:20, 20104214.42B/s] 24%|██▍ | 126369792/526681800 [00:06<00:18, 21147630.57B/s] 24%|██▍ | 128536576/526681800 [00:06<00:19, 20392018.28B/s] 25%|██▍ | 130780160/526681800 [00:06<00:18, 20847580.98B/s] 25%|██▌ | 133204992/526681800 [00:06<00:18, 21391543.60B/s] 26%|██▌ | 135370752/526681800 [00:06<00:18, 21047086.38B/s] 26%|██▌ | 137760768/526681800 [00:06<00:17, 21828635.20B/s] 27%|██▋ | 139965440/526681800 [00:07<00:18, 20556287.33B/s] 27%|██▋ | 142462976/526681800 [00:07<00:17, 21708489.88B/s] 27%|██▋ | 144672768/526681800 [00:07<00:18, 20355616.49B/s] 28%|██▊

In [72]:
data = tensorflow_datasets.load('glue/mrpc')

INFO:absl:Load pre-computed datasetinfo (eg: splits) from bucket.
INFO:absl:Loading info from GCS for glue/mrpc/0.0.2
INFO:absl:Generating dataset glue (/root/tensorflow_datasets/glue/mrpc/0.0.2)
 Downloading and preparing dataset glue (1.43 MiB) to /root/tensorflow_datasets/glue/mrpc/0.0.2... 
Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]INFO:absl:Downloading https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2Fmrpc_dev_ids.tsv?alt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc into /root/tensorflow_datasets/downloads/fire.goog.com_v0_b_mtl-sent-repr.apps.com_o_2FjSIMlCiqs1QSmIykr4IRPnEHjPuGwAz5i40v8K9U0Z8.tsvalt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc.tmp.a2894c73354f4e7e868edd44e0e01ec9...
Dl Completed...: 0%| | 0/1 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]INFO:absl:Downloading https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt into /root/tensorflow_datasets/downloads/dl.fbaip.com_sente_sente_msr_parap_trainfGxPZuQWGBti4Tbd1YNOwQr-OqxPejJ7gcp0Al6mlSk.txt.tmp.5df8a9158db84131a1ca68864942ff5c...
Dl Completed...: 0%| | 0/2 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]INFO:absl:Downloading https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_test.txt into /root/tensorflow_datasets/downloads/dl.fbaip.com_sente_sente_msr_parap_test0PdekMcyqYR-w4Rx_d7OTryq0J3RlYRn4rAMajy9Mak.txt.tmp.5a7b6a7559dd4b338ff17ee0b0244026...
Dl Completed...: 0%| | 0/3 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]/databricks/python/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
 InsecureRequestWarning)
/databricks/python/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
 InsecureRequestWarning)
/databricks/python/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
 InsecureRequestWarning)
Dl Completed...: 0%| | 0/3 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]Dl Completed...: 33%|███▎ | 1/3 [00:00<00:00, 3.45 url/s]Dl Completed...: 33%|███▎ | 1/3 [00:00<00:00, 3.45 url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]Dl Completed...: 33%|███▎ | 1/3 [00:00<00:00, 3.45 url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]Dl Completed...: 33%|███▎ | 1/3 [00:00<00:00, 3.45 url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]Dl Completed...: 67%|██████▋ | 2/3 [00:00<00:00, 3.85 url/s]Dl Completed...: 67%|██████▋ | 2/3 [00:00<00:00, 3.85 url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]Dl Completed...: 100%|██████████| 3/3 [00:00<00:00, 3.85 url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 100%|██████████| 3/3 [00:00<00:00, 5.75 url/s]
INFO:absl:Generating split train
INFO:absl:Writing TFRecords
0 examples [00:00, ? examples/s]249 examples [00:00, 2488.31 examples/s]508 examples [00:00, 2515.48 examples/s]758 examples [00:00, 2509.05 examples/s]1016 examples [00:00, 2528.87 examples/s]1252 examples [00:00, 2472.75 examples/s]1505 examples [00:00, 2487.97 examples/s]1771 examples [00:00, 2535.99 examples/s]2021 examples [00:00, 2522.47 examples/s]2278 examples [00:00, 2533.58 examples/s]2536 examples [00:01, 2545.02 examples/s]2790 examples [00:01, 2543.13 examples/s]3040 examples [00:01, 2529.81 examples/s]3303 examples [00:01, 2556.36 examples/s]3572 examples [00:01, 2594.86 examples/s] Shuffling...: 0%| | 0/1 [00:00<?, ? shard/s]WARNING:tensorflow:From /databricks/python/lib/python3.7/sit

In [73]:
# Prepare dataset for GLUE as a tf.data.Dataset instance
train_dataset = glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, task='mrpc')
valid_dataset = glue_convert_examples_to_features(data['validation'], tokenizer, max_length=128, task='mrpc')
train_dataset = train_dataset.shuffle(100).batch(32).repeat(2)
valid_dataset = valid_dataset.batch(64)

In [74]:
# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule 
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [75]:
# Train and evaluate using tf.keras.Model.fit()
history = model.fit(train_dataset, epochs=2, steps_per_epoch=115,
                    validation_data=valid_dataset, validation_steps=7)

Train for 115 steps, validate for 7 steps
Epoch 1/2
 1/115 [..............................] - ETA: 49:53 - loss: 0.7010 - accuracy: 0.4688 2/115 [..............................] - ETA: 26:07 - loss: 0.7182 - accuracy: 0.4531 3/115 [..............................] - ETA: 18:09 - loss: 0.7013 - accuracy: 0.5000 4/115 [>.............................] - ETA: 14:09 - loss: 0.7084 - accuracy: 0.4922 5/115 [>.............................] - ETA: 11:45 - loss: 0.7028 - accuracy: 0.5000 6/115 [>.............................] - ETA: 10:08 - loss: 0.6947 - accuracy: 0.5260 7/115 [>.............................] - ETA: 8:59 - loss: 0.6879 - accuracy: 0.5446  8/115 [=>............................] - ETA: 8:06 - loss: 0.6856 - accuracy: 0.5586 9/115 [=>............................] - ETA: 7:25 - loss: 0.6878 - accuracy: 0.5590 10/115 [=>............................] - ETA: 6:52 - loss: 0.6745 - accuracy: 0.5813 11/115 [=>............................] - ETA: 6:24 - loss: 0.6715 - accuracy: 0.5881 12/115 [==>...........................] - ETA: 6:02 - loss: 0.6774 - accuracy: 0.5833 13/115 [==>...........................] - ETA: 5:42 - loss: 0.6725 - accuracy: 0.5938 14/115 [==>...........................] - ETA: 5:25 - loss: 0.6720 - accuracy: 0.5960 15/115 [==>...........................] - ETA: 5:10 - loss: 0.6681 - accuracy: 0.6000 16/115 [===>..........................] - ETA: 4:57 - loss: 0.6619 - accuracy: 0.6113 17/115 [===>..........................] - ETA: 4:46 - loss: 0.6644 - accuracy: 0.6085 18/115 [===>..........................] - ETA: 4:35 - loss: 0.6636 - accuracy: 0.6094 19/115 [===>..........................] - ETA: 4:25 - loss: 0.6615 - accuracy: 0.6151 20/115 [====>.........................] - ETA: 4:16 - loss: 0.6574 - accuracy: 0.6203 21/115 [====>.........................] - ETA: 4:08 - loss: 0.6530 - accuracy: 0.6250 22/115 [====>.........................] - ETA: 4:01 - loss: 0.6473 - accuracy: 0.6307 23/115 [=====>........................] - ETA: 3:53 - loss: 0.6427 - accuracy: 0.6345

In [76]:
model.save_pretrained('/tmp/mrpc')

In [77]:
tokenizer.save_pretrained('/tmp/mrpc')

Out[33]: ('/tmp/mrpc/vocab.txt',
 '/tmp/mrpc/special_tokens_map.json',
 '/tmp/mrpc/added_tokens.json')

In [78]:
%fs
cp -r file:/tmp/mrpc dbfs:/ml/judith/transformers/mrpc/1

res20: Boolean = true

In [79]:
%fs
ls dbfs:/ml/judith/transformers/mrpc

path,name,size
dbfs:/ml/judith/transformers/mrpc/1/,1/,0
dbfs:/ml/judith/transformers/mrpc/2/,2/,0
dbfs:/ml/judith/transformers/mrpc/config.json,config.json,543


In [80]:
model1 = TFBertForSequenceClassification.from_pretrained("/tmp/mrpc")

In [81]:
sentence_0 = "This research was consistent with his findings."
sentence_1 = "His findings were compatible with this research."
sentence_2 = "His findings were not compatible with this research."
inputs_1 = tokenizer.encode_plus(sentence_0, sentence_1, add_special_tokens=True, return_tensors='tf')
inputs_2 = tokenizer.encode_plus(sentence_0, sentence_2, add_special_tokens=True, return_tensors='tf')

In [82]:
model1(inputs_1)[0]

Out[42]: <tf.Tensor: id=73262, shape=(1, 2), dtype=float32, numpy=array([[-1.7221127, 2.6670885]], dtype=float32)>

In [83]:
model1(inputs_2)[0]

Out[43]: <tf.Tensor: id=75431, shape=(1, 2), dtype=float32, numpy=array([[-1.6031767, 2.5245368]], dtype=float32)>

How to download GLUE dataset?

In [85]:
%sh
curl https://gist.githubusercontent.com/W4ngatang/60c2bdb54d156a41194446737ce03e2e/raw/17b8dd0d724281ed7c3b2aeeda662b92809aadd5/download_glue_data.py --output /tmp/download_glue_data.py

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
100 8225 100 8225 0 0 25427 0 --:--:-- --:--:-- --:--:-- 25464

In [86]:
%fs
cp -r file:/databricks/driver/glue_data dbfs:/ml/judith/datasets/glue_data

res10: Boolean = true

In [87]:
%fs
ls dbfs:/ml/judith/datasets/glue_data

path,name,size
dbfs:/ml/judith/datasets/glue_data/CoLA/,CoLA/,0
dbfs:/ml/judith/datasets/glue_data/MNLI/,MNLI/,0
dbfs:/ml/judith/datasets/glue_data/MRPC/,MRPC/,0
dbfs:/ml/judith/datasets/glue_data/QNLI/,QNLI/,0
dbfs:/ml/judith/datasets/glue_data/QQP/,QQP/,0
dbfs:/ml/judith/datasets/glue_data/RTE/,RTE/,0
dbfs:/ml/judith/datasets/glue_data/SNLI/,SNLI/,0
dbfs:/ml/judith/datasets/glue_data/SST-2/,SST-2/,0
dbfs:/ml/judith/datasets/glue_data/STS-B/,STS-B/,0
dbfs:/ml/judith/datasets/glue_data/WNLI/,WNLI/,0


In [88]:
%sh
/databricks/python/bin/python -V
/databricks/python/bin/python /tmp/download_glue_data.py

Python 3.7.3
Downloading and extracting CoLA...
	Completed!
Downloading and extracting SST...
	Completed!
Processing MRPC...
Local MRPC data not specified, downloading data from https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt
	Completed!
Downloading and extracting QQP...
	Completed!
Downloading and extracting STS...
	Completed!
Downloading and extracting MNLI...
	Completed!
Downloading and extracting SNLI...
	Completed!
Downloading and extracting QNLI...
	Completed!
Downloading and extracting RTE...
	Completed!
Downloading and extracting WNLI...
	Completed!
Downloading and extracting diagnostic...
	Completed!